In [1]:
import json
import random
import re


from itertools import islice

import datazimmer as dz
import pandas as pd
from parquetranger import TableRepo
from tqdm.notebook import tqdm
from unidecode import unidecode

from src.whoscored_parsing.extractors import *
from src.whoscored_parsing.t2_integrators import *
from src.whoscored_collect import *
from src.raw_tables import event_table

In [2]:
pcevs = list(islice(tqdm(project.depot.get_handler_events(GetMatchDetails)), 100))

99it [00:00, 989.48it/s]


In [ ]:
misses = []
all_form_uses = []
all_formations = []
all_events = []
att_dfs = [[] for _ in table_extractor_pairs]
for pind, pcev in tqdm([*enumerate(random.sample(pcevs, len(pcevs)))]):
    try:
        pcev_content = json.loads(pcev.content or "null")
        if pcev_content is None or pcev_content.get("home") is None:
            raise ValueError()
    except (json.JSONDecodeError, ValueError):
        misses.append(pcev)
        continue
    url_dic = parse_url(pcev.url)
    all_events += get_event_records(pcev_content, url_dic)
    rest = {**url_dic, **pcev_content}
    form_use_list, forms_list = handle_formations(rest)
    all_form_uses += form_use_list
    all_formations += forms_list
    for (_, extractor, df_formatter), dfl in zip(
        table_extractor_pairs, att_dfs
    ):
        out = extractor.parse(rest)
        if isinstance(out, dict):
            out = [out]
        _df = pd.DataFrame(out)
        if extractor.id_key is not None:
            if extractor.id_key in _df.columns:
                _df = _df.set_index(extractor.id_key)
            else:
                print("no index ", extractor.id_key)
                print(pcev.url)
                continue
        dfl.append(_df.pipe(df_formatter))

    if (pind + 1) % 500 == 0:
        fix_event_df(all_events).pipe(event_table.extend)
        all_events = []

fix_event_df(all_events).pipe(event_table.extend)

pd.DataFrame(all_form_uses).pipe(formation_use_table.extend)
pd.DataFrame(all_formations).drop_duplicates(subset="formation_id").set_index(
    "formation_id"
).pipe(formation_table.replace_records)

for (trepo, _, __), dfl in zip(table_extractor_pairs, att_dfs):
    _df = pd.concat(dfl)

    if _df.index.name is None:
        _df.pipe(trepo.extend)
    else:
        _df.pipe(trepo.replace_records)